In [2]:
import numpy as np 
import pandas as pd 
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler 
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.metrics import mean_absolute_error

In [3]:
window_size = 25
train = pd.read_csv("./data/train.csv")
stocks = range(0,200)
train = train.sort_values(['date_id','time_id'])
train['wap'] = train['wap'].fillna(train['wap'].mean())
scaler =  MinMaxScaler()
targetScaler = MinMaxScaler()
train['wap_scaled'] = scaler.fit_transform(train['wap'].to_numpy().reshape(-1, 1)).squeeze()
train['target_scaled'] = targetScaler.fit_transform(train['target'].to_numpy().reshape(-1, 1)).squeeze()
table = pd.pivot_table(train,values=['wap_scaled','target','seconds_in_bucket'],columns=['stock_id'],index=['date_id','time_id'])


table.fillna(method='bfill',inplace=True)
table.fillna(method='ffill',inplace=True)

print(table.head())
table.isna().sum()

                seconds_in_bucket                                            \
stock_id                      0     1     2     3     4     5     6     7     
date_id time_id                                                               
0       0                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
        1                    10.0  10.0  10.0  10.0  10.0  10.0  10.0  10.0   
        2                    20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0   
        3                    30.0  30.0  30.0  30.0  30.0  30.0  30.0  30.0   
        4                    40.0  40.0  40.0  40.0  40.0  40.0  40.0  40.0   

                             ... wap_scaled                                \
stock_id          8     9    ...        190       191       192       193   
date_id time_id              ...                                            
0       0         0.0   0.0  ...   0.443856  0.443856  0.443856  0.443856   
        1        10.0  10.0  ...   0.443970  0.451954  0.44

C:\Users\adams\AppData\Local\Temp\ipykernel_19828\2711329630.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='bfill',inplace=True)
C:\Users\adams\AppData\Local\Temp\ipykernel_19828\2711329630.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='ffill',inplace=True)


                   stock_id
seconds_in_bucket  0           0
                   1           0
                   2           0
                   3           0
                   4           0
                              ..
wap_scaled         195         0
                   196         0
                   197         0
                   198         0
                   199         0
Length: 600, dtype: int64

In [4]:
rolling_target = {}
for stock in stocks:
    rolling_target[stock] = table['target'][stock].rolling(30).mean()
    rolling_target[stock] = rolling_target[stock].fillna(0)
    rolling_target[stock].iloc[:1000]

rolling_target_df = pd.DataFrame(rolling_target)
print(rolling_target_df.head())


                 0    1    2    3    4    5    6    7    8    9    ...  190  \
date_id time_id                                                    ...        
0       0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                 191  192  193  194  195  196  197  198  199  
date_id time_id                                               
0       0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 r

In [5]:
train.corr()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,wap_scaled,target_scaled
stock_id,1.000000e+00,5.028770e-03,-1.116655e-16,0.024639,0.004067,-0.000752,0.035130,-0.000831,0.004736,0.004324,0.032975,-0.006342,0.027150,-0.000629,0.000130,0.005029,0.568940,-0.000629,0.000130
date_id,5.028770e-03,1.000000e+00,1.280965e-14,0.006441,-0.010572,-0.006456,0.000525,-0.003004,-0.016150,-0.006528,0.024776,-0.006917,0.018673,-0.006839,0.000893,0.999998,0.446700,-0.006839,0.000893
seconds_in_bucket,-1.116655e-16,1.280965e-14,1.000000e+00,-0.123946,-0.003635,-0.000835,0.068625,-0.000550,0.017627,0.014819,0.162867,-0.017826,0.146338,-0.001259,-0.002203,0.002083,0.218802,-0.001259,-0.002203
imbalance_size,2.463857e-02,6.441422e-03,-1.239456e-01,1.000000,-0.010159,-0.003717,0.512367,0.000523,-0.030932,0.005691,0.021141,-0.012774,0.022623,-0.003495,-0.001190,0.006183,-0.005419,-0.003495,-0.001190
imbalance_buy_sell_flag,4.066622e-03,-1.057243e-02,-3.634734e-03,-0.010159,1.000000,0.182100,-0.006003,0.018010,0.508853,0.129579,0.035238,0.132520,-0.013352,0.133822,0.015102,-0.010580,0.001548,0.133822,0.015102
reference_price,-7.519350e-04,-6.456302e-03,-8.347507e-04,-0.003717,0.182100,1.000000,0.007724,0.015349,0.473595,0.983646,0.007271,0.985068,-0.006528,0.988113,-0.017394,-0.006458,-0.006632,0.988113,-0.017394
matched_size,3.513044e-02,5.254090e-04,6.862495e-02,0.512367,-0.006003,0.007724,1.000000,-0.000705,-0.002722,0.023564,0.078554,-0.007270,0.079209,0.008281,0.000197,0.000668,0.037265,0.008281,0.000197
far_price,-8.311000e-04,-3.004471e-03,-5.499413e-04,0.000523,0.018010,0.015349,-0.000705,1.000000,0.031519,0.014327,0.002229,0.014671,-0.000153,0.014643,-0.001804,-0.003005,-0.001236,0.014643,-0.001804
near_price,4.736243e-03,-1.615006e-02,1.762744e-02,-0.030932,0.508853,0.473595,-0.002722,0.031519,1.000000,0.458373,0.032190,0.460380,-0.013298,0.462579,0.006045,-0.016133,-0.005470,0.462579,0.006045
bid_price,4.324366e-03,-6.528025e-03,1.481858e-02,0.005691,0.129579,0.983646,0.023564,0.014327,0.458373,1.000000,-0.001098,0.972906,-0.008456,0.989277,-0.022326,-0.006497,0.000849,0.989277,-0.022326
